In [1]:
import napari
import numpy as np
import matplotlib.pyplot as plt
import z5py
from pathlib import Path

from cryofib.n5_utils import read_volume, print_key_tree
from cryofib.data_loaders import load_platynereis_pred_n5

## View n5 datasets of platynereis membrane segmentation

In [2]:
data_dir = Path("/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane")
n5_paths = list(data_dir.glob("*.n5"))
f_n5_list = [z5py.File(n5_path, "r") for n5_path in n5_paths]

In [3]:
print(n5_paths)

[PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_01.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_03.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_04.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_05.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_06.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_07.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_08.n5'), PosixPath('/g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_09.n5')]


In [4]:
print_key_tree(f_n5_list[0])

Key structure of z5 file /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_01.n5
volumes
volumes/labels
volumes/labels/segmentation
volumes/labels/segmentation/s1
volumes/raw
volumes/raw/s0
volumes/raw/s1
volumes/raw/s2
volumes/raw/s3
3dunet
3dunet/raw
3dunet/labels


## Load predictions

In [14]:
f_pred_list = load_platynereis_pred_n5()

## Read volumes

In [22]:
idx = 1
roi = np.s_[:]
raw_stacks = []
for s in range(4):
    raw_stacks.append(read_volume(f_n5_list[idx], ("volumes/raw/s%d"%s), roi))

segm = read_volume(f_n5_list[idx], "volumes/labels/segmentation/s1", roi)

Reading roi slice(None, None, None) of volume volumes/raw/s0 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5
Read volume with shape (130, 1256, 1256), data type uint8
Reading roi slice(None, None, None) of volume volumes/raw/s1 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5
Read volume with shape (130, 628, 628), data type uint8
Reading roi slice(None, None, None) of volume volumes/raw/s2 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5
Read volume with shape (193, 826, 826), data type uint8
Reading roi slice(None, None, None) of volume volumes/raw/s3 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5
Read volume with shape (97, 413, 413), data type uint8
Reading roi slice(None, None, None) of volume volumes/labels/segmentation/s1 from /g/kreshuk/buglakova/data/platynereis_em_membranes/membrane/train_data_membrane_02.n5

In [23]:
print_key_tree(f_pred_list[1])

Key structure of z5 file /scratch/buglakova/data/platynereis_em_membranes/membrane/predictions/prediction_membrane_01.n5
predictions
predictions/full
predictions/full/bg
predictions/full/fg
predictions/full/boundaries


In [24]:
boundary_pred = read_volume(f_pred_list[idx], "predictions/full/boundaries", roi)

Reading roi slice(None, None, None) of volume predictions/full/boundaries from /scratch/buglakova/data/platynereis_em_membranes/membrane/predictions/prediction_membrane_01.n5
Read volume with shape (130, 628, 628), data type float32


## Open it in napari

In [25]:
v = napari.Viewer()

### Open untransformed fluorescence

In [26]:
resolution = [25, 20, 20]
# for s in range(4):
#     v.add_image(raw_stacks[s], name=("s%d"%s), scale=resolution)
v.add_image(raw_stacks[1], name="s1", scale=resolution)
v.add_labels(segm, name="segmentation", scale=resolution)

<Labels layer 'segmentation' at 0x7f844c466c50>

In [27]:
v.add_image(boundary_pred, name="3D unet prediction", scale=resolution, blending="additive", colormap="red")

<Image layer '3D unet prediction' at 0x7f84c4835600>